In [2]:
import s3fs
import numpy as np
import pandas as pd
from pandas import set_option
from matplotlib import pyplot as plt
from pandas import read_csv 

In [3]:
#view data (Providers_Updated_430.txt)
df = pd.read_csv('s3://filtered-datasets/Providers_Updated_430.txt', sep='\t')
from pandas import set_option
set_option('display.max_columns', None)
df.head(5)


/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (71) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_city,nppes_provider_zip5,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,ge65_suppress_flag,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65_suppress_flag,bene_count_ge65,brand_suppress_flag,brand_claim_count,brand_drug_cost,generic_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,other_claim_count,other_drug_cost,mapd_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_suppress_flag,pdp_claim_count,pdp_drug_cost,lis_suppress_flag,lis_claim_count,lis_drug_cost,nonlis_suppress_flag,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,opioid_bene_count,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antibiotic_claim_count,antibiotic_drug_cost,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_bene_ge65_suppress_flg,antipsych_bene_count_ge65,average_age_of_beneficiaries,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score,EXCLYear,REINYear,excl_type,exclusion_flag,country,state,nppes_credentials
0,1003000126,ENKESHAFI,ARDALAN,NaN,M,I,900 SETON DR,CUMBERLAND,21502.0,Internal Medicine,S,E,677.0,695.5,32639.57,14788.0,234.0,not_suppressed,516.0,532.533333,23174.78,11428.0,not_suppressed,182.0,#,NaN,NaN,not_suppressed,552.0,5941.53,*,NaN,NaN,not_suppressed,150.0,3032.66,not_suppressed,527.0,29606.91,not_suppressed,305.0,17267.72,not_suppressed,372.0,15371.85,25.0,197.47,134.0,24.0,3.69,5.0,NaN,NaN,5.0,NaN,125.0,2478.65,104.0,*,NaN,NaN,*,NaN,72.0,142.0,92.0,143.0,91.0,2.1685,NaN,NaN,NaN,0,US,MD,Medical Doctor
1,1003000142,KHALIL,RASHID,NaN,M,I,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623.0,Surgery,S,E,1946.0,2054.8,140189.01,58605.0,276.0,not_suppressed,881.0,919.066667,50838.50,26089.0,not_suppressed,160.0,not_suppressed,235.0,106048.28,not_suppressed,1711.0,34140.73,not_suppressed,0.0,0.0,not_suppressed,930.0,67081.22,not_suppressed,1016.0,73107.79,not_suppressed,1184.0,94361.07,not_suppressed,762.0,45827.94,1040.0,50895.21,29238.0,200.0,53.44,200.0,37637.86,5853.0,49.0,19.23,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,65.0,184.0,92.0,143.0,133.0,1.8029,NaN,NaN,NaN,0,US,OH,Medical Doctor
2,1003000167,ESCOBAR,JULIO,E,M,I,5 PINE CONE RD,DAYTON,89403.0,Dentist,S,N,55.0,55.0,302.01,430.0,33.0,#,NaN,NaN,NaN,NaN,#,NaN,not_suppressed,0.0,0.00,#,NaN,NaN,*,NaN,NaN,not_suppressed,16.0,58.19,not_suppressed,39.0,243.82,*,NaN,NaN,#,NaN,NaN,14.0,61.95,35.0,11.0,25.45,0.0,0.00,0.0,0.0,0.00,28.0,134.92,22.0,NaN,0.0,0.0,NaN,0.0,72.0,16.0,17.0,NaN,NaN,1.0598,NaN,NaN,NaN,0,US,NV,Dentist/DDS
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,F,I,322 N AZUSA AVE STE 202,LA PUENTE,91744.0,Dentist,S,N,18.0,18.0,113.81,150.0,5.0,#,NaN,NaN,NaN,NaN,#,NaN,not_suppressed,0.0,0.00,not_suppressed,18.0,113.81,not_suppressed,0.0,0.0,*,NaN,NaN,*,NaN,NaN,#,NaN,NaN,*,NaN,NaN,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,12.0,81.54,5.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,US,CA,Dentist/DDS
4,1003000282,BLAKEMORE,ROSIE,K,F,I,TENNESSEE PRISON FOR WOMEN,NASHVILLE,37243.0,Nurse,S,N,90.0,110.0,7561.21,2681.0,11.0,not_suppressed,65.0,85.000000,3894.08,1931.0,*,NaN,not_suppressed,28.0,6576.28,not_suppressed,62.0,984.93,not_suppressed,0.0,0.0,not_suppressed,22.0,485.04,not_suppressed,68.0,7076.17,not_suppressed,73.0,3947.56,not_suppressed,17.0,3613.65,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,62.0,NaN,NaN,NaN,NaN,4.5148,NaN,NaN,NaN,0,US,TN,Nurse


In [4]:
df.shape

(1162898, 76)

In [5]:
#display null value counts for each column
pd.set_option('display.max_rows', 120)
df.isnull().sum()

npi                                       0
nppes_provider_last_org_name             34
nppes_provider_first_name                22
nppes_provider_mi                    361527
nppes_provider_gender                     3
nppes_entity_code                         0
nppes_provider_street1                    1
nppes_provider_city                       0
nppes_provider_zip5                      52
specialty_description                     0
description_flag                          0
medicare_prvdr_enroll_status              0
total_claim_count                         0
total_30_day_fill_count                   0
total_drug_cost                           0
total_day_supply                          0
bene_count                               78
ge65_suppress_flag                        0
total_claim_count_ge65               252456
total_30_day_fill_count_ge65         252456
total_drug_cost_ge65                 252456
total_day_supply_ge65                252456
bene_count_ge65_suppress_flag   

In [6]:
# imputation for total_claim_count_ge65

# total_claim_count_ge65 (If ge65_suppress_flag = *=>total_claim_count_ge65 = 5)
df['total_claim_count_ge65'] = np.where((df['ge65_suppress_flag'] == '*'), 5, df['total_claim_count_ge65'])

#If ge65_suppress_flag = #=>total_claim_count_ge65 =  total_claim_count – 5
df['total_claim_count_ge65'] = np.where((df['ge65_suppress_flag'] == '#'), df['total_claim_count'] - 5, 
                                          df['total_claim_count_ge65'])

df.head(5)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_city,nppes_provider_zip5,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,ge65_suppress_flag,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65_suppress_flag,bene_count_ge65,brand_suppress_flag,brand_claim_count,brand_drug_cost,generic_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,other_claim_count,other_drug_cost,mapd_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_suppress_flag,pdp_claim_count,pdp_drug_cost,lis_suppress_flag,lis_claim_count,lis_drug_cost,nonlis_suppress_flag,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,opioid_bene_count,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antibiotic_claim_count,antibiotic_drug_cost,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_bene_ge65_suppress_flg,antipsych_bene_count_ge65,average_age_of_beneficiaries,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score,EXCLYear,REINYear,excl_type,exclusion_flag,country,state,nppes_credentials
0,1003000126,ENKESHAFI,ARDALAN,NaN,M,I,900 SETON DR,CUMBERLAND,21502.0,Internal Medicine,S,E,677.0,695.5,32639.57,14788.0,234.0,not_suppressed,516.0,532.533333,23174.78,11428.0,not_suppressed,182.0,#,NaN,NaN,not_suppressed,552.0,5941.53,*,NaN,NaN,not_suppressed,150.0,3032.66,not_suppressed,527.0,29606.91,not_suppressed,305.0,17267.72,not_suppressed,372.0,15371.85,25.0,197.47,134.0,24.0,3.69,5.0,NaN,NaN,5.0,NaN,125.0,2478.65,104.0,*,NaN,NaN,*,NaN,72.0,142.0,92.0,143.0,91.0,2.1685,NaN,NaN,NaN,0,US,MD,Medical Doctor
1,1003000142,KHALIL,RASHID,NaN,M,I,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623.0,Surgery,S,E,1946.0,2054.8,140189.01,58605.0,276.0,not_suppressed,881.0,919.066667,50838.50,26089.0,not_suppressed,160.0,not_suppressed,235.0,106048.28,not_suppressed,1711.0,34140.73,not_suppressed,0.0,0.0,not_suppressed,930.0,67081.22,not_suppressed,1016.0,73107.79,not_suppressed,1184.0,94361.07,not_suppressed,762.0,45827.94,1040.0,50895.21,29238.0,200.0,53.44,200.0,37637.86,5853.0,49.0,19.23,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,65.0,184.0,92.0,143.0,133.0,1.8029,NaN,NaN,NaN,0,US,OH,Medical Doctor
2,1003000167,ESCOBAR,JULIO,E,M,I,5 PINE CONE RD,DAYTON,89403.0,Dentist,S,N,55.0,55.0,302.01,430.0,33.0,#,50.0,NaN,NaN,NaN,#,NaN,not_suppressed,0.0,0.00,#,NaN,NaN,*,NaN,NaN,not_suppressed,16.0,58.19,not_suppressed,39.0,243.82,*,NaN,NaN,#,NaN,NaN,14.0,61.95,35.0,11.0,25.45,0.0,0.00,0.0,0.0,0.00,28.0,134.92,22.0,NaN,0.0,0.0,NaN,0.0,72.0,16.0,17.0,NaN,NaN,1.0598,NaN,NaN,NaN,0,US,NV,Dentist/DDS
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,F,I,322 N AZUSA AVE STE 202,LA PUENTE,91744.0,Dentist,S,N,18.0,18.0,113.81,150.0,5.0,#,13.0,NaN,NaN,NaN,#,NaN,not_suppressed,0.0,0.00,not_suppressed,18.0,113.81,not_suppressed,0.0,0.0,*,NaN,NaN,*,NaN,NaN,#,NaN,NaN,*,NaN,NaN,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,12.0,81.54,5.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,US,CA,Dentist/DDS
4,1003000282,BLAKEMORE,ROSIE,K,F,I,TENNESSEE PRISON FOR WOMEN,NASHVILLE,37243.0,Nurse,S,N,90.0,110.0,7561.21,2681.0,11.0,not_suppressed,65.0,85.000000,3894.08,1931.0,*,NaN,not_suppressed,28.0,6576.28,not_suppressed,62.0,984.93,not_suppressed,0.0,0.0,not_suppressed,22.0,485.04,not_suppressed,68.0,7076.17,not_suppressed,73.0,3947.56,not_suppressed,17.0,3613.65,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,62.0,NaN,NaN,NaN,NaN,4.5148,NaN,NaN,NaN,0,US,TN,Nurse


In [7]:
# imputation for bene_count_ge65

# Bene_count_ge65 ( If bene_count_ge65_suppress_flag = * =>bene_count_ge65 = 5)
df['bene_count_ge65'] = np.where((df['bene_count_ge65_suppress_flag'] == '*'), 5, df['bene_count_ge65'])

# If bene_count_ge65_suppress_flag = # =>bene_count_ge65 = total_bene_count – 5
df['bene_count_ge65'] = np.where((df['bene_count_ge65_suppress_flag'] == '#'), df['bene_count'] - 5, 
                                          df['bene_count_ge65'])
df.head(5)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_city,nppes_provider_zip5,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,ge65_suppress_flag,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65_suppress_flag,bene_count_ge65,brand_suppress_flag,brand_claim_count,brand_drug_cost,generic_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,other_claim_count,other_drug_cost,mapd_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_suppress_flag,pdp_claim_count,pdp_drug_cost,lis_suppress_flag,lis_claim_count,lis_drug_cost,nonlis_suppress_flag,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,opioid_bene_count,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antibiotic_claim_count,antibiotic_drug_cost,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_bene_ge65_suppress_flg,antipsych_bene_count_ge65,average_age_of_beneficiaries,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score,EXCLYear,REINYear,excl_type,exclusion_flag,country,state,nppes_credentials
0,1003000126,ENKESHAFI,ARDALAN,NaN,M,I,900 SETON DR,CUMBERLAND,21502.0,Internal Medicine,S,E,677.0,695.5,32639.57,14788.0,234.0,not_suppressed,516.0,532.533333,23174.78,11428.0,not_suppressed,182.0,#,NaN,NaN,not_suppressed,552.0,5941.53,*,NaN,NaN,not_suppressed,150.0,3032.66,not_suppressed,527.0,29606.91,not_suppressed,305.0,17267.72,not_suppressed,372.0,15371.85,25.0,197.47,134.0,24.0,3.69,5.0,NaN,NaN,5.0,NaN,125.0,2478.65,104.0,*,NaN,NaN,*,NaN,72.0,142.0,92.0,143.0,91.0,2.1685,NaN,NaN,NaN,0,US,MD,Medical Doctor
1,1003000142,KHALIL,RASHID,NaN,M,I,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623.0,Surgery,S,E,1946.0,2054.8,140189.01,58605.0,276.0,not_suppressed,881.0,919.066667,50838.50,26089.0,not_suppressed,160.0,not_suppressed,235.0,106048.28,not_suppressed,1711.0,34140.73,not_suppressed,0.0,0.0,not_suppressed,930.0,67081.22,not_suppressed,1016.0,73107.79,not_suppressed,1184.0,94361.07,not_suppressed,762.0,45827.94,1040.0,50895.21,29238.0,200.0,53.44,200.0,37637.86,5853.0,49.0,19.23,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,65.0,184.0,92.0,143.0,133.0,1.8029,NaN,NaN,NaN,0,US,OH,Medical Doctor
2,1003000167,ESCOBAR,JULIO,E,M,I,5 PINE CONE RD,DAYTON,89403.0,Dentist,S,N,55.0,55.0,302.01,430.0,33.0,#,50.0,NaN,NaN,NaN,#,28.0,not_suppressed,0.0,0.00,#,NaN,NaN,*,NaN,NaN,not_suppressed,16.0,58.19,not_suppressed,39.0,243.82,*,NaN,NaN,#,NaN,NaN,14.0,61.95,35.0,11.0,25.45,0.0,0.00,0.0,0.0,0.00,28.0,134.92,22.0,NaN,0.0,0.0,NaN,0.0,72.0,16.0,17.0,NaN,NaN,1.0598,NaN,NaN,NaN,0,US,NV,Dentist/DDS
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,F,I,322 N AZUSA AVE STE 202,LA PUENTE,91744.0,Dentist,S,N,18.0,18.0,113.81,150.0,5.0,#,13.0,NaN,NaN,NaN,#,0.0,not_suppressed,0.0,0.00,not_suppressed,18.0,113.81,not_suppressed,0.0,0.0,*,NaN,NaN,*,NaN,NaN,#,NaN,NaN,*,NaN,NaN,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,12.0,81.54,5.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,US,CA,Dentist/DDS
4,1003000282,BLAKEMORE,ROSIE,K,F,I,TENNESSEE PRISON FOR WOMEN,NASHVILLE,37243.0,Nurse,S,N,90.0,110.0,7561.21,2681.0,11.0,not_suppressed,65.0,85.000000,3894.08,1931.0,*,5.0,not_suppressed,28.0,6576.28,not_suppressed,62.0,984.93,not_suppressed,0.0,0.0,not_suppressed,22.0,485.04,not_suppressed,68.0,7076.17,not_suppressed,73.0,3947.56,not_suppressed,17.0,3613.65,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,62.0,NaN,NaN,NaN,NaN,4.5148,NaN,NaN,NaN,0,US,TN,Nurse


In [8]:
# imputations for brand claim count,generic_claim_count, and other claim count

# brand claim count (If brand_suppress_flag  = *=>brand_claim_count = 5)
df['brand_claim_count'] = np.where((df['brand_suppress_flag'] == '*'), 5, df['brand_claim_count'])

# generic_claim_count (If generic_suppress_flag = *=>generic_claim_count = 5)
df['generic_claim_count'] = np.where((df['generic_suppress_flag'] == '*'), 5, df['generic_claim_count'])

# other_claim_count(If other_suppress_flag = *=>other_claim_count = 5)
df['other_claim_count'] = np.where((df['other_suppress_flag'] == '*'), 5, df['other_claim_count'])

# brand_claim_count(If brand_suppress_flag = #=>brand_claim_count =  total_claim_count - other_claim_count – generic_claim_count)
df['brand_claim_count'] = np.where((df['brand_suppress_flag'] == '#'), df['total_claim_count'] - df['other_claim_count'] -df['generic_claim_count'], 
                                   df['brand_claim_count'])

# generic claim count (If generic_suppress_flag = #=>generic_claim_count = total_claim_count – brand_claim_count – other_claim_count)
df['generic_claim_count'] = np.where((df['generic_suppress_flag'] == '#'), df['total_claim_count'] - df['brand_claim_count'] -df['other_claim_count'], 
                                   df['generic_claim_count'])

# other claim count (If other_suppress_flag = #=>other_claim_count = total_claim_count – brand_claim_count – generic_claim_count)
df['other_claim_count'] = np.where((df['other_suppress_flag'] == '#'), df['total_claim_count'] - df['brand_claim_count'] -df['generic_claim_count'], 
                                   df['other_claim_count'])

df.head(5)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_city,nppes_provider_zip5,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,ge65_suppress_flag,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65_suppress_flag,bene_count_ge65,brand_suppress_flag,brand_claim_count,brand_drug_cost,generic_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,other_claim_count,other_drug_cost,mapd_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_suppress_flag,pdp_claim_count,pdp_drug_cost,lis_suppress_flag,lis_claim_count,lis_drug_cost,nonlis_suppress_flag,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,opioid_bene_count,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antibiotic_claim_count,antibiotic_drug_cost,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_bene_ge65_suppress_flg,antipsych_bene_count_ge65,average_age_of_beneficiaries,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score,EXCLYear,REINYear,excl_type,exclusion_flag,country,state,nppes_credentials
0,1003000126,ENKESHAFI,ARDALAN,NaN,M,I,900 SETON DR,CUMBERLAND,21502.0,Internal Medicine,S,E,677.0,695.5,32639.57,14788.0,234.0,not_suppressed,516.0,532.533333,23174.78,11428.0,not_suppressed,182.0,#,120.0,NaN,not_suppressed,552.0,5941.53,*,5.0,NaN,not_suppressed,150.0,3032.66,not_suppressed,527.0,29606.91,not_suppressed,305.0,17267.72,not_suppressed,372.0,15371.85,25.0,197.47,134.0,24.0,3.69,5.0,NaN,NaN,5.0,NaN,125.0,2478.65,104.0,*,NaN,NaN,*,NaN,72.0,142.0,92.0,143.0,91.0,2.1685,NaN,NaN,NaN,0,US,MD,Medical Doctor
1,1003000142,KHALIL,RASHID,NaN,M,I,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623.0,Surgery,S,E,1946.0,2054.8,140189.01,58605.0,276.0,not_suppressed,881.0,919.066667,50838.50,26089.0,not_suppressed,160.0,not_suppressed,235.0,106048.28,not_suppressed,1711.0,34140.73,not_suppressed,0.0,0.0,not_suppressed,930.0,67081.22,not_suppressed,1016.0,73107.79,not_suppressed,1184.0,94361.07,not_suppressed,762.0,45827.94,1040.0,50895.21,29238.0,200.0,53.44,200.0,37637.86,5853.0,49.0,19.23,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,65.0,184.0,92.0,143.0,133.0,1.8029,NaN,NaN,NaN,0,US,OH,Medical Doctor
2,1003000167,ESCOBAR,JULIO,E,M,I,5 PINE CONE RD,DAYTON,89403.0,Dentist,S,N,55.0,55.0,302.01,430.0,33.0,#,50.0,NaN,NaN,NaN,#,28.0,not_suppressed,0.0,0.00,#,50.0,NaN,*,5.0,NaN,not_suppressed,16.0,58.19,not_suppressed,39.0,243.82,*,NaN,NaN,#,NaN,NaN,14.0,61.95,35.0,11.0,25.45,0.0,0.00,0.0,0.0,0.00,28.0,134.92,22.0,NaN,0.0,0.0,NaN,0.0,72.0,16.0,17.0,NaN,NaN,1.0598,NaN,NaN,NaN,0,US,NV,Dentist/DDS
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,F,I,322 N AZUSA AVE STE 202,LA PUENTE,91744.0,Dentist,S,N,18.0,18.0,113.81,150.0,5.0,#,13.0,NaN,NaN,NaN,#,0.0,not_suppressed,0.0,0.00,not_suppressed,18.0,113.81,not_suppressed,0.0,0.0,*,NaN,NaN,*,NaN,NaN,#,NaN,NaN,*,NaN,NaN,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,12.0,81.54,5.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,US,CA,Dentist/DDS
4,1003000282,BLAKEMORE,ROSIE,K,F,I,TENNESSEE PRISON FOR WOMEN,NASHVILLE,37243.0,Nurse,S,N,90.0,110.0,7561.21,2681.0,11.0,not_suppressed,65.0,85.000000,3894.08,1931.0,*,5.0,not_suppressed,28.0,6576.28,not_suppressed,62.0,984.93,not_suppressed,0.0,0.0,not_suppressed,22.0,485.04,not_suppressed,68.0,7076.17,not_suppressed,73.0,3947.56,not_suppressed,17.0,3613.65,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,62.0,NaN,NaN,NaN,NaN,4.5148,NaN,NaN,NaN,0,US,TN,Nurse


In [21]:
# imputations for mapd claim count and Pdp_claim_count

# mapd claim count (If mapd_suppress_flag = *=>Mapd_claim_count = 5)
df['mapd_claim_count'] = np.where((df['mapd_suppress_flag'] == '*'), 5, df['mapd_claim_count'])

# Pdp_claim_count (If pdp_suppress_flag = *=>Pdp_claim_count = 5)
df['pdp_claim_count'] = np.where((df['pdp_suppress_flag'] == '*'), 5, df['pdp_claim_count'])

# mapd claim count (If mapd_suppress_flag = #=>Mapd_claim_count = total_claim_count – pdp_claim_count)
df['mapd_claim_count'] = np.where((df['mapd_suppress_flag'] == '#'), df['total_claim_count'] - df['pdp_claim_count'], 
                                   df['mapd_claim_count'])

# pdp claim count (If pdp_suppress_flag = #=>Pdp_suppress_flag = total_claim_count – mapd_claim_count)
df['pdp_claim_count'] = np.where((df['pdp_suppress_flag'] == '#'), df['total_claim_count'] - df['mapd_claim_count'], 
                                   df['pdp_claim_count'])
df.head(10)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_city,nppes_provider_zip5,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,ge65_suppress_flag,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65_suppress_flag,bene_count_ge65,brand_suppress_flag,brand_claim_count,brand_drug_cost,generic_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,other_claim_count,other_drug_cost,mapd_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_suppress_flag,pdp_claim_count,pdp_drug_cost,lis_suppress_flag,lis_claim_count,lis_drug_cost,nonlis_suppress_flag,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,opioid_bene_count,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antibiotic_claim_count,antibiotic_drug_cost,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_bene_ge65_suppress_flg,antipsych_bene_count_ge65,average_age_of_beneficiaries,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score,EXCLYear,REINYear,excl_type,exclusion_flag,country,state,nppes_credentials
0,1003000126,ENKESHAFI,ARDALAN,NaN,M,I,900 SETON DR,CUMBERLAND,21502.0,Internal Medicine,S,E,677.0,695.5,32639.57,14788.0,234.0,not_suppressed,516.0,532.533333,23174.780,11428.0,not_suppressed,182.0,#,120.0,17122.30,not_suppressed,552.0,5941.53,*,5.0,0.00,not_suppressed,150.0,3032.660,not_suppressed,527.0,29606.91,not_suppressed,305.0,17267.72,not_suppressed,372.0,15371.85,25.0,197.47,134.0,24.0,3.69,5.0,0.00,0.0,5.0,0.00,125.0,2478.65,104.0,*,5.0,0.00,*,5.0,72.0,142.0,92.0,143.0,91.0,2.1685,NaN,NaN,NaN,0,US,MD,Medical Doctor
1,1003000142,KHALIL,RASHID,NaN,M,I,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623.0,Surgery,S,E,1946.0,2054.8,140189.01,58605.0,276.0,not_suppressed,881.0,919.066667,50838.500,26089.0,not_suppressed,160.0,not_suppressed,235.0,106048.28,not_suppressed,1711.0,34140.73,not_suppressed,0.0,0.00,not_suppressed,930.0,67081.220,not_suppressed,1016.0,73107.79,not_suppressed,1184.0,94361.07,not_suppressed,762.0,45827.94,1040.0,50895.21,29238.0,200.0,53.44,200.0,37637.86,5853.0,49.0,19.23,0.0,0.00,0.0,NaN,0.0,0.00,NaN,0.0,65.0,184.0,92.0,143.0,133.0,1.8029,NaN,NaN,NaN,0,US,OH,Medical Doctor
2,1003000167,ESCOBAR,JULIO,E,M,I,5 PINE CONE RD,DAYTON,89403.0,Dentist,S,N,55.0,55.0,302.01,430.0,33.0,#,50.0,306.000000,13928.695,6920.0,#,28.0,not_suppressed,0.0,0.00,#,50.0,8144.09,*,5.0,0.00,not_suppressed,16.0,58.190,not_suppressed,39.0,243.82,*,5.0,7822.22,#,50.0,8329.63,14.0,61.95,35.0,11.0,25.45,0.0,0.00,0.0,0.0,0.00,28.0,134.92,22.0,NaN,0.0,0.00,NaN,0.0,72.0,16.0,17.0,89.0,40.0,1.0598,NaN,NaN,NaN,0,US,NV,Dentist/DDS
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,F,I,322 N AZUSA AVE STE 202,LA PUENTE,91744.0,Dentist,S,N,18.0,18.0,113.81,150.0,5.0,#,13.0,306.000000,13928.695,6920.0,#,0.0,not_suppressed,0.0,0.00,not_suppressed,18.0,113.81,not_suppressed,0.0,0.00,*,5.0,5038.525,*,5.0,11189.15,#,73.0,7822.22,*,5.0,8329.63,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,12.0,81.54,5.0,NaN,0.0,0.00,NaN,0.0,71.0,73.0,47.0,89.0,40.0,1.3465,NaN,NaN,NaN,0,US,CA,Dentist/DDS
4,1003000282,BLAKEMORE,ROSIE,K,F,I,TENNESSEE PRISON FOR WOMEN,NASHVILLE,37243.0,Nurse,S,N,90.0,110.0,7561.21,2681.0,11.0,not_suppressed,65.0,85.000000,3894.080,1931.0,*,5.0,not_suppressed,28.0,6576.28,not_suppressed,62.0,984.93,not_suppressed,0.0,0.00,not_suppressed,22.0,485.040,not_suppressed,68.0,7076.17,not_suppressed,73.0,3947.56,not_suppressed,17.0,3613.65,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,NaN,0.0,0.00,NaN,0.0,62.0,73.0,47.0,89.0,40.0,4.5148,NaN,NaN,NaN,0,US,TN,Nurse
5,1003

In [10]:
# imputations for lis claim count and nonlis claim count

# lis claim count (If lis_suppress_flag = * =>lis_claim_count = 5)
df['lis_claim_count'] = np.where((df['lis_suppress_flag'] == '*'), 5, df['lis_claim_count'])

# nonlis claim count (If nonlis_suppress_flag = *=>ononlis_claim_count = 5)
df['nonlis_claim_count'] = np.where((df['nonlis_suppress_flag'] == '*'), 5, df['nonlis_claim_count'])

# lis claim count if lis_suppress_flag = #=> lis_claim_count = total_claim_count – non_claim_count 
# non_claim_count is missing in the data
#imputing remaining null values in lis claim count with the median for the lis claim count column
df['lis_claim_count']=df['lis_claim_count'].fillna(df['lis_claim_count'].median())

# nonlis claim count (If nonlis_suppress_flag = # =>non_suppress_flag = total_claim_count – lis_claim_count)
df['nonlis_claim_count'] = np.where((df['nonlis_suppress_flag'] == '#'), df['total_claim_count'] - df['lis_claim_count'], 
                                   df['nonlis_claim_count'])

df.head(15)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_city,nppes_provider_zip5,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,ge65_suppress_flag,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65_suppress_flag,bene_count_ge65,brand_suppress_flag,brand_claim_count,brand_drug_cost,generic_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,other_claim_count,other_drug_cost,mapd_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_suppress_flag,pdp_claim_count,pdp_drug_cost,lis_suppress_flag,lis_claim_count,lis_drug_cost,nonlis_suppress_flag,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,opioid_bene_count,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antibiotic_claim_count,antibiotic_drug_cost,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_bene_ge65_suppress_flg,antipsych_bene_count_ge65,average_age_of_beneficiaries,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score,EXCLYear,REINYear,excl_type,exclusion_flag,country,state,nppes_credentials
0,1003000126,ENKESHAFI,ARDALAN,NaN,M,I,900 SETON DR,CUMBERLAND,21502.0,Internal Medicine,S,E,677.0,695.5,32639.57,14788.0,234.0,not_suppressed,516.0,532.533333,23174.78,11428.0,not_suppressed,182.0,#,120.0,NaN,not_suppressed,552.0,5941.53,*,5.0,NaN,not_suppressed,150.0,3032.66,not_suppressed,527.0,29606.91,not_suppressed,305.0,17267.72,not_suppressed,372.0,15371.85,25.0,197.47,134.0,24.0,3.69,5.0,NaN,NaN,5.0,NaN,125.0,2478.65,104.0,*,NaN,NaN,*,NaN,72.0,142.0,92.0,143.0,91.0,2.1685,NaN,NaN,NaN,0,US,MD,Medical Doctor
1,1003000142,KHALIL,RASHID,NaN,M,I,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623.0,Surgery,S,E,1946.0,2054.8,140189.01,58605.0,276.0,not_suppressed,881.0,919.066667,50838.50,26089.0,not_suppressed,160.0,not_suppressed,235.0,106048.28,not_suppressed,1711.0,34140.73,not_suppressed,0.0,0.00,not_suppressed,930.0,67081.22,not_suppressed,1016.0,73107.79,not_suppressed,1184.0,94361.07,not_suppressed,762.0,45827.94,1040.0,50895.21,29238.0,200.0,53.44,200.0,37637.86,5853.0,49.0,19.23,0.0,0.00,0.0,NaN,0.0,0.00,NaN,0.0,65.0,184.0,92.0,143.0,133.0,1.8029,NaN,NaN,NaN,0,US,OH,Medical Doctor
2,1003000167,ESCOBAR,JULIO,E,M,I,5 PINE CONE RD,DAYTON,89403.0,Dentist,S,N,55.0,55.0,302.01,430.0,33.0,#,50.0,NaN,NaN,NaN,#,28.0,not_suppressed,0.0,0.00,#,50.0,NaN,*,5.0,NaN,not_suppressed,16.0,58.19,not_suppressed,39.0,243.82,*,5.0,NaN,#,50.0,NaN,14.0,61.95,35.0,11.0,25.45,0.0,0.00,0.0,0.0,0.00,28.0,134.92,22.0,NaN,0.0,0.00,NaN,0.0,72.0,16.0,17.0,NaN,NaN,1.0598,NaN,NaN,NaN,0,US,NV,Dentist/DDS
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,F,I,322 N AZUSA AVE STE 202,LA PUENTE,91744.0,Dentist,S,N,18.0,18.0,113.81,150.0,5.0,#,13.0,NaN,NaN,NaN,#,0.0,not_suppressed,0.0,0.00,not_suppressed,18.0,113.81,not_suppressed,0.0,0.00,*,NaN,NaN,*,NaN,NaN,#,73.0,NaN,*,5.0,NaN,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,12.0,81.54,5.0,NaN,0.0,0.00,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,US,CA,Dentist/DDS
4,1003000282,BLAKEMORE,ROSIE,K,F,I,TENNESSEE PRISON FOR WOMEN,NASHVILLE,37243.0,Nurse,S,N,90.0,110.0,7561.21,2681.0,11.0,not_suppressed,65.0,85.000000,3894.08,1931.0,*,5.0,not_suppressed,28.0,6576.28,not_suppressed,62.0,984.93,not_suppressed,0.0,0.00,not_suppressed,22.0,485.04,not_suppressed,68.0,7076.17,not_suppressed,73.0,3947.56,not_suppressed,17.0,3613.65,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,0.0,0.00,0.0,NaN,0.0,0.00,NaN,0.0,62.0,NaN,NaN,NaN,NaN,4.5148,NaN,NaN,NaN,0,US,TN,Nurse
5,1003000407,GIRARDI,DAVID,J,M,I,100 HOSPITAL RD,BROOKVILLE,15825.0,General/Family Practice,S,E,2788.

In [18]:
# imputations for antipsych claim count ge65

# antipsych claim count ge65 (If antipsych_ge65_suppress_flag = *=>antipsych_claim_count_ge65 = 5)
df['antipsych_claim_count_ge65'] = np.where((df['antipsych_ge65_suppress_flag'] == '*'), 5, df['antipsych_claim_count_ge65'])

# antipsych claim count ge65 (If antipsych_ge65_suppress_flag = #)
#impute remaining null values with the median for the antipsych claim count ge65 
df['antipsych_claim_count_ge65']=df['antipsych_claim_count_ge65'].fillna(df['antipsych_claim_count_ge65'].median()) 

df.head(10)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_city,nppes_provider_zip5,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,ge65_suppress_flag,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65_suppress_flag,bene_count_ge65,brand_suppress_flag,brand_claim_count,brand_drug_cost,generic_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,other_claim_count,other_drug_cost,mapd_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_suppress_flag,pdp_claim_count,pdp_drug_cost,lis_suppress_flag,lis_claim_count,lis_drug_cost,nonlis_suppress_flag,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,opioid_bene_count,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antibiotic_claim_count,antibiotic_drug_cost,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_bene_ge65_suppress_flg,antipsych_bene_count_ge65,average_age_of_beneficiaries,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score,EXCLYear,REINYear,excl_type,exclusion_flag,country,state,nppes_credentials
0,1003000126,ENKESHAFI,ARDALAN,NaN,M,I,900 SETON DR,CUMBERLAND,21502.0,Internal Medicine,S,E,677.0,695.5,32639.57,14788.0,234.0,not_suppressed,516.0,532.533333,23174.780,11428.0,not_suppressed,182.0,#,120.0,17122.30,not_suppressed,552.0,5941.53,*,5.0,0.00,not_suppressed,150.0,3032.660,not_suppressed,527.0,29606.91,not_suppressed,305.0,17267.72,not_suppressed,372.0,15371.85,25.0,197.47,134.0,24.0,3.69,5.0,0.00,0.0,5.0,0.00,125.0,2478.65,104.0,*,5.0,0.00,*,5.0,72.0,142.0,92.0,143.0,91.0,2.1685,NaN,NaN,NaN,0,US,MD,Medical Doctor
1,1003000142,KHALIL,RASHID,NaN,M,I,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623.0,Surgery,S,E,1946.0,2054.8,140189.01,58605.0,276.0,not_suppressed,881.0,919.066667,50838.500,26089.0,not_suppressed,160.0,not_suppressed,235.0,106048.28,not_suppressed,1711.0,34140.73,not_suppressed,0.0,0.00,not_suppressed,930.0,67081.220,not_suppressed,1016.0,73107.79,not_suppressed,1184.0,94361.07,not_suppressed,762.0,45827.94,1040.0,50895.21,29238.0,200.0,53.44,200.0,37637.86,5853.0,49.0,19.23,0.0,0.00,0.0,NaN,0.0,0.00,NaN,0.0,65.0,184.0,92.0,143.0,133.0,1.8029,NaN,NaN,NaN,0,US,OH,Medical Doctor
2,1003000167,ESCOBAR,JULIO,E,M,I,5 PINE CONE RD,DAYTON,89403.0,Dentist,S,N,55.0,55.0,302.01,430.0,33.0,#,50.0,306.000000,13928.695,6920.0,#,28.0,not_suppressed,0.0,0.00,#,50.0,8144.09,*,5.0,0.00,not_suppressed,16.0,58.190,not_suppressed,39.0,243.82,*,5.0,7822.22,#,50.0,8329.63,14.0,61.95,35.0,11.0,25.45,0.0,0.00,0.0,0.0,0.00,28.0,134.92,22.0,NaN,0.0,0.00,NaN,0.0,72.0,16.0,17.0,89.0,40.0,1.0598,NaN,NaN,NaN,0,US,NV,Dentist/DDS
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,F,I,322 N AZUSA AVE STE 202,LA PUENTE,91744.0,Dentist,S,N,18.0,18.0,113.81,150.0,5.0,#,13.0,306.000000,13928.695,6920.0,#,0.0,not_suppressed,0.0,0.00,not_suppressed,18.0,113.81,not_suppressed,0.0,0.00,*,5.0,5038.525,*,5.0,11189.15,#,73.0,7822.22,*,5.0,8329.63,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,12.0,81.54,5.0,NaN,0.0,0.00,NaN,0.0,71.0,73.0,47.0,89.0,40.0,1.3465,NaN,NaN,NaN,0,US,CA,Dentist/DDS
4,1003000282,BLAKEMORE,ROSIE,K,F,I,TENNESSEE PRISON FOR WOMEN,NASHVILLE,37243.0,Nurse,S,N,90.0,110.0,7561.21,2681.0,11.0,not_suppressed,65.0,85.000000,3894.080,1931.0,*,5.0,not_suppressed,28.0,6576.28,not_suppressed,62.0,984.93,not_suppressed,0.0,0.00,not_suppressed,22.0,485.040,not_suppressed,68.0,7076.17,not_suppressed,73.0,3947.56,not_suppressed,17.0,3613.65,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,NaN,0.0,0.00,NaN,0.0,62.0,73.0,47.0,89.0,40.0,4.5148,NaN,NaN,NaN,0,US,TN,Nurse
5,1003

In [19]:
# imputations for antipsych_bene_count_ge65

# antipsych_bene_count_ge65 (If antipsych_bene_ge65_suppress_flag = * =>antipsych_bene_count_ge65 = 5)
df['antipsych_bene_count_ge65'] = np.where((df['antipsych_bene_ge65_suppress_flg'] == '*'), 5, df['antipsych_bene_count_ge65'])

# antipsych_bene_count_ge65 (If antipsych_bene_ge65_suppress_flag = #) 
# impute remaining null values with the median for the antipsych_bene_count_ge65  column
df['antipsych_bene_count_ge65']=df['antipsych_bene_count_ge65'].fillna(df['antipsych_bene_count_ge65'].median())

df.head(10)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_city,nppes_provider_zip5,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,ge65_suppress_flag,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65_suppress_flag,bene_count_ge65,brand_suppress_flag,brand_claim_count,brand_drug_cost,generic_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,other_claim_count,other_drug_cost,mapd_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_suppress_flag,pdp_claim_count,pdp_drug_cost,lis_suppress_flag,lis_claim_count,lis_drug_cost,nonlis_suppress_flag,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,opioid_bene_count,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antibiotic_claim_count,antibiotic_drug_cost,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_bene_ge65_suppress_flg,antipsych_bene_count_ge65,average_age_of_beneficiaries,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score,EXCLYear,REINYear,excl_type,exclusion_flag,country,state,nppes_credentials
0,1003000126,ENKESHAFI,ARDALAN,NaN,M,I,900 SETON DR,CUMBERLAND,21502.0,Internal Medicine,S,E,677.0,695.5,32639.57,14788.0,234.0,not_suppressed,516.0,532.533333,23174.780,11428.0,not_suppressed,182.0,#,120.0,17122.30,not_suppressed,552.0,5941.53,*,5.0,0.00,not_suppressed,150.0,3032.660,not_suppressed,527.0,29606.91,not_suppressed,305.0,17267.72,not_suppressed,372.0,15371.85,25.0,197.47,134.0,24.0,3.69,5.0,0.00,0.0,5.0,0.00,125.0,2478.65,104.0,*,5.0,0.00,*,5.0,72.0,142.0,92.0,143.0,91.0,2.1685,NaN,NaN,NaN,0,US,MD,Medical Doctor
1,1003000142,KHALIL,RASHID,NaN,M,I,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623.0,Surgery,S,E,1946.0,2054.8,140189.01,58605.0,276.0,not_suppressed,881.0,919.066667,50838.500,26089.0,not_suppressed,160.0,not_suppressed,235.0,106048.28,not_suppressed,1711.0,34140.73,not_suppressed,0.0,0.00,not_suppressed,930.0,67081.220,not_suppressed,1016.0,73107.79,not_suppressed,1184.0,94361.07,not_suppressed,762.0,45827.94,1040.0,50895.21,29238.0,200.0,53.44,200.0,37637.86,5853.0,49.0,19.23,0.0,0.00,0.0,NaN,0.0,0.00,NaN,0.0,65.0,184.0,92.0,143.0,133.0,1.8029,NaN,NaN,NaN,0,US,OH,Medical Doctor
2,1003000167,ESCOBAR,JULIO,E,M,I,5 PINE CONE RD,DAYTON,89403.0,Dentist,S,N,55.0,55.0,302.01,430.0,33.0,#,50.0,306.000000,13928.695,6920.0,#,28.0,not_suppressed,0.0,0.00,#,50.0,8144.09,*,5.0,0.00,not_suppressed,16.0,58.190,not_suppressed,39.0,243.82,*,5.0,7822.22,#,50.0,8329.63,14.0,61.95,35.0,11.0,25.45,0.0,0.00,0.0,0.0,0.00,28.0,134.92,22.0,NaN,0.0,0.00,NaN,0.0,72.0,16.0,17.0,89.0,40.0,1.0598,NaN,NaN,NaN,0,US,NV,Dentist/DDS
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,F,I,322 N AZUSA AVE STE 202,LA PUENTE,91744.0,Dentist,S,N,18.0,18.0,113.81,150.0,5.0,#,13.0,306.000000,13928.695,6920.0,#,0.0,not_suppressed,0.0,0.00,not_suppressed,18.0,113.81,not_suppressed,0.0,0.00,*,5.0,5038.525,*,5.0,11189.15,#,73.0,7822.22,*,5.0,8329.63,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,12.0,81.54,5.0,NaN,0.0,0.00,NaN,0.0,71.0,73.0,47.0,89.0,40.0,1.3465,NaN,NaN,NaN,0,US,CA,Dentist/DDS
4,1003000282,BLAKEMORE,ROSIE,K,F,I,TENNESSEE PRISON FOR WOMEN,NASHVILLE,37243.0,Nurse,S,N,90.0,110.0,7561.21,2681.0,11.0,not_suppressed,65.0,85.000000,3894.080,1931.0,*,5.0,not_suppressed,28.0,6576.28,not_suppressed,62.0,984.93,not_suppressed,0.0,0.00,not_suppressed,22.0,485.040,not_suppressed,68.0,7076.17,not_suppressed,73.0,3947.56,not_suppressed,17.0,3613.65,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,NaN,0.0,0.00,NaN,0.0,62.0,73.0,47.0,89.0,40.0,4.5148,NaN,NaN,NaN,0,US,TN,Nurse
5,1003

In [22]:
#display all remaining null value counts
pd.set_option('display.max_rows', 120)
df.isnull().sum()

npi                                       0
nppes_provider_last_org_name             34
nppes_provider_first_name                22
nppes_provider_mi                    361527
nppes_provider_gender                     3
nppes_entity_code                         0
nppes_provider_street1                    1
nppes_provider_city                       0
nppes_provider_zip5                      52
specialty_description                     0
description_flag                          0
medicare_prvdr_enroll_status              0
total_claim_count                         0
total_30_day_fill_count                   0
total_drug_cost                           0
total_day_supply                          0
bene_count                                0
ge65_suppress_flag                        0
total_claim_count_ge65                    0
total_30_day_fill_count_ge65              0
total_drug_cost_ge65                      0
total_day_supply_ge65                     0
bene_count_ge65_suppress_flag   

In [23]:
from pandas import set_option
set_option('display.max_columns', None)
df.head(10)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_city,nppes_provider_zip5,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,ge65_suppress_flag,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65_suppress_flag,bene_count_ge65,brand_suppress_flag,brand_claim_count,brand_drug_cost,generic_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,other_claim_count,other_drug_cost,mapd_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_suppress_flag,pdp_claim_count,pdp_drug_cost,lis_suppress_flag,lis_claim_count,lis_drug_cost,nonlis_suppress_flag,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,opioid_bene_count,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antibiotic_claim_count,antibiotic_drug_cost,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_bene_ge65_suppress_flg,antipsych_bene_count_ge65,average_age_of_beneficiaries,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score,EXCLYear,REINYear,excl_type,exclusion_flag,country,state,nppes_credentials
0,1003000126,ENKESHAFI,ARDALAN,NaN,M,I,900 SETON DR,CUMBERLAND,21502.0,Internal Medicine,S,E,677.0,695.5,32639.57,14788.0,234.0,not_suppressed,516.0,532.533333,23174.780,11428.0,not_suppressed,182.0,#,120.0,17122.30,not_suppressed,552.0,5941.53,*,5.0,0.00,not_suppressed,150.0,3032.660,not_suppressed,527.0,29606.91,not_suppressed,305.0,17267.72,not_suppressed,372.0,15371.85,25.0,197.47,134.0,24.0,3.69,5.0,0.00,0.0,5.0,0.00,125.0,2478.65,104.0,*,5.0,0.00,*,5.0,72.0,142.0,92.0,143.0,91.0,2.1685,NaN,NaN,NaN,0,US,MD,Medical Doctor
1,1003000142,KHALIL,RASHID,NaN,M,I,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623.0,Surgery,S,E,1946.0,2054.8,140189.01,58605.0,276.0,not_suppressed,881.0,919.066667,50838.500,26089.0,not_suppressed,160.0,not_suppressed,235.0,106048.28,not_suppressed,1711.0,34140.73,not_suppressed,0.0,0.00,not_suppressed,930.0,67081.220,not_suppressed,1016.0,73107.79,not_suppressed,1184.0,94361.07,not_suppressed,762.0,45827.94,1040.0,50895.21,29238.0,200.0,53.44,200.0,37637.86,5853.0,49.0,19.23,0.0,0.00,0.0,NaN,0.0,0.00,NaN,0.0,65.0,184.0,92.0,143.0,133.0,1.8029,NaN,NaN,NaN,0,US,OH,Medical Doctor
2,1003000167,ESCOBAR,JULIO,E,M,I,5 PINE CONE RD,DAYTON,89403.0,Dentist,S,N,55.0,55.0,302.01,430.0,33.0,#,50.0,306.000000,13928.695,6920.0,#,28.0,not_suppressed,0.0,0.00,#,50.0,8144.09,*,5.0,0.00,not_suppressed,16.0,58.190,not_suppressed,39.0,243.82,*,5.0,7822.22,#,50.0,8329.63,14.0,61.95,35.0,11.0,25.45,0.0,0.00,0.0,0.0,0.00,28.0,134.92,22.0,NaN,0.0,0.00,NaN,0.0,72.0,16.0,17.0,89.0,40.0,1.0598,NaN,NaN,NaN,0,US,NV,Dentist/DDS
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,F,I,322 N AZUSA AVE STE 202,LA PUENTE,91744.0,Dentist,S,N,18.0,18.0,113.81,150.0,5.0,#,13.0,306.000000,13928.695,6920.0,#,0.0,not_suppressed,0.0,0.00,not_suppressed,18.0,113.81,not_suppressed,0.0,0.00,*,5.0,5038.525,*,5.0,11189.15,#,73.0,7822.22,*,5.0,8329.63,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,12.0,81.54,5.0,NaN,0.0,0.00,NaN,0.0,71.0,73.0,47.0,89.0,40.0,1.3465,NaN,NaN,NaN,0,US,CA,Dentist/DDS
4,1003000282,BLAKEMORE,ROSIE,K,F,I,TENNESSEE PRISON FOR WOMEN,NASHVILLE,37243.0,Nurse,S,N,90.0,110.0,7561.21,2681.0,11.0,not_suppressed,65.0,85.000000,3894.080,1931.0,*,5.0,not_suppressed,28.0,6576.28,not_suppressed,62.0,984.93,not_suppressed,0.0,0.00,not_suppressed,22.0,485.040,not_suppressed,68.0,7076.17,not_suppressed,73.0,3947.56,not_suppressed,17.0,3613.65,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,NaN,0.0,0.00,NaN,0.0,62.0,73.0,47.0,89.0,40.0,4.5148,NaN,NaN,NaN,0,US,TN,Nurse
5,1003

In [20]:
# impute other null values

# total_30_day_fill_count_ge65         
df['total_30_day_fill_count_ge65']=df['total_30_day_fill_count_ge65'].fillna(df['total_30_day_fill_count_ge65'].median())

# bene_count 
df['bene_count']=df['bene_count'].fillna(df['bene_count'].median())

# total_drug_cost_ge65   
df['total_drug_cost_ge65']=df['total_drug_cost_ge65'].fillna(df['total_drug_cost_ge65'].median())

# total_day_supply_ge65   
df['total_day_supply_ge65']=df['total_day_supply_ge65'].fillna(df['total_day_supply_ge65'].median())

# bene_count_ge65 
df['bene_count_ge65']=df['bene_count_ge65'].fillna(df['bene_count_ge65'].median())

# brand_drug_cost                      
df['brand_drug_cost']=df['brand_drug_cost'].fillna(df['brand_drug_cost'].median())

# generic_drug_cost                    
df['generic_drug_cost']=df['generic_drug_cost'].fillna(df['generic_drug_cost'].median())

# other_drug_cost                      
df['other_drug_cost']=df['other_drug_cost'].fillna(df['other_drug_cost'].median())

# mapd_drug_cost                       
df['mapd_drug_cost']=df['mapd_drug_cost'].fillna(df['mapd_drug_cost'].median())

# pdp_drug_cost                        
df['pdp_drug_cost']=df['pdp_drug_cost'].fillna(df['pdp_drug_cost'].median())

# lis_claim_count                      
df['lis_claim_count']=df['lis_claim_count'].fillna(df['lis_claim_count'].median())

# lis_drug_cost                        
df['lis_drug_cost']=df['lis_drug_cost'].fillna(df['lis_drug_cost'].median())

# nonlis_claim_count                   
df['nonlis_claim_count']=df['nonlis_claim_count'].fillna(df['nonlis_claim_count'].median())

# nonlis_drug_cost                     
df['nonlis_drug_cost']=df['nonlis_drug_cost'].fillna(df['nonlis_drug_cost'].median())

# opioid_claim_count                       
df['opioid_claim_count']=df['opioid_claim_count'].fillna(df['opioid_claim_count'].median())

# opioid_drug_cost                     
df['opioid_drug_cost']=df['opioid_drug_cost'].fillna(df['opioid_drug_cost'].median())

# opioid_day_supply  
df['opioid_day_supply']=df['opioid_day_supply'].fillna(df['opioid_day_supply'].median())

# opioid_bene_count 
df['opioid_bene_count']=df['opioid_bene_count'].fillna(df['opioid_bene_count'].median())

# opioid_prescriber_rate               
df['opioid_prescriber_rate']=df['opioid_prescriber_rate'].fillna(df['opioid_prescriber_rate'].median())

# la_opioid_claim_count                    
df['la_opioid_claim_count']=df['la_opioid_claim_count'].fillna(df['la_opioid_claim_count'].median())

# la_opioid_drug_cost                  
df['la_opioid_drug_cost']=df['la_opioid_drug_cost'].fillna(df['la_opioid_drug_cost'].median())

# la_opioid_day_supply                 
df['la_opioid_day_supply']=df['la_opioid_day_supply'].fillna(df['la_opioid_day_supply'].median())

# la_opioid_bene_count 
df['la_opioid_bene_count']=df['la_opioid_bene_count'].fillna(df['la_opioid_bene_count'].median())

# la_opioid_prescriber_rate            
df['la_opioid_prescriber_rate']=df['la_opioid_prescriber_rate'].fillna(df['la_opioid_prescriber_rate'].median())

# antibiotic_claim_count                  
df['antibiotic_claim_count']=df['antibiotic_claim_count'].fillna(df['antibiotic_claim_count'].median())

# antibiotic_drug_cost                 
df['antibiotic_drug_cost']=df['antibiotic_drug_cost'].fillna(df['antibiotic_drug_cost'].median())

# antibiotic_bene_count                   
df['antibiotic_bene_count']=df['antibiotic_bene_count'].fillna(df['antibiotic_bene_count'].median())

# antipsych_drug_cost_ge65             
df['antipsych_drug_cost_ge65']=df['antipsych_drug_cost_ge65'].fillna(df['antipsych_drug_cost_ge65'].median())

# average_age_of_beneficiaries         
df['average_age_of_beneficiaries']=df['average_age_of_beneficiaries'].fillna(df['average_age_of_beneficiaries'].median())

# beneficiary_female_count             
df['beneficiary_female_count']=df['beneficiary_female_count'].fillna(df['beneficiary_female_count'].median())

# beneficiary_male_count               
df['beneficiary_male_count']=df['beneficiary_male_count'].fillna(df['beneficiary_male_count'].median())

# beneficiary_nondual_count            
df['beneficiary_nondual_count']=df['beneficiary_nondual_count'].fillna(df['beneficiary_nondual_count'].median())

# beneficiary_dual_count               
df['beneficiary_dual_count']=df['beneficiary_dual_count'].fillna(df['beneficiary_dual_count'].median())

# beneficiary_average_risk_score       
df['beneficiary_average_risk_score']=df['beneficiary_average_risk_score'].fillna(df['beneficiary_average_risk_score'].median())

In [25]:
#display all remaining null value counts
pd.set_option('display.max_rows', 120)
df.isnull().sum()

npi                                       0
nppes_provider_last_org_name             34
nppes_provider_first_name                22
nppes_provider_mi                    361527
nppes_provider_gender                     3
nppes_entity_code                         0
nppes_provider_street1                    1
nppes_provider_city                       0
nppes_provider_zip5                      52
specialty_description                     0
description_flag                          0
medicare_prvdr_enroll_status              0
total_claim_count                         0
total_30_day_fill_count                   0
total_drug_cost                           0
total_day_supply                          0
bene_count                                0
ge65_suppress_flag                        0
total_claim_count_ge65                    0
total_30_day_fill_count_ge65              0
total_drug_cost_ge65                      0
total_day_supply_ge65                     0
bene_count_ge65_suppress_flag   

In [27]:
# convert EXCL Year and Excl type to objects
df["EXCLYear"]= df["EXCLYear"].astype(object)
#convert nppes provider zip to object
df["REINYear"]= df["REINYear"].astype(object)


Further analysis will be performed to ascertain needed further processing including transformations or column (atrribute) dropping. 